# Calculate Sunset and Sunrise
Welcome to this ugly notebook! What you find here: Calculates the earliest and latest sunrise and sunset for:
- the current year
- if everything was in wintertime
- if everything was in summertime

Earliest sunrise is not during longest day. There may be a fancy formula to calculate this. We go bruth force. So we calculate the sunrise and sunset for each day in year 2025 and find the corresponding entry.

In [1]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
from astral import Observer
from astral.sun import sun
from astral.location import Location, LocationInfo
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from zoneinfo import ZoneInfo
import pytz
from tqdm import tqdm
import json
import consts

## Define

In [2]:
USE_GRID = 'hex'

## Import Data

In [3]:
if USE_GRID == 'nuts':
    gdf_grid = gpd.read_file(consts.PATH_MASTERNUTS)
    gdf_grid['geometry'] = gdf_grid['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))
    PATH_TIMEDATA = consts.PATH_TIMEDATA_NUTS
elif USE_GRID == 'hex':
    gdf_grid = gpd.read_file(consts.PATH_HEXAGON)
    PATH_TIMEDATA = consts.PATH_TIMEDATA_HEX
else:
    raise ValueError(f"Unknown grid type {USE_GRID}")

## Helpers

In [4]:
def transform_date_to_winter_summer_time(dt, local_tz):

    datetime_local = dt.astimezone(local_tz)

    datetime_winter = datetime_local.time()
    datetime_summer = datetime_local.time()
    if bool(datetime_local.dst()):
        datetime_winter = (datetime_local - timedelta(hours=1)).time()
    else:
        datetime_summer = (datetime_local + timedelta(hours=1)).time()

    return {
            'utc': dt,
            'local': datetime_local,
            'time_winter': datetime_winter,
            'time_summer': datetime_summer,
        }

def find_earliest_and_latest_in_list(suns_per_day, event, timeperiod):
    return {
            'earliest': min(suns_per_day[event], key=lambda x: x[timeperiod])[timeperiod],
            'earliest_day': min(suns_per_day[event], key=lambda x: x[timeperiod])['local'].date(),
            'latest': max(suns_per_day[event], key=lambda x: x[timeperiod])[timeperiod],
            'latest_day': max(suns_per_day[event], key=lambda x: x[timeperiod])['local'].date()
        }

## Calculate Time of Sunrise and Sunset

In [5]:
# Get all possible dates from 2025
dates = [datetime(2025, 1, 1) + timedelta(days=x) for x in range(365)]

timedata = []

with tqdm(total=len(gdf_grid)) as pbar:
    for i, row in gdf_grid.iterrows():

        pbar.update()

        location = Observer(row.geometry.centroid.y, row.geometry.centroid.x)

        # Earliest sunrise is not the same as the longest day. We need to calculate
        # the earliest day. We could do that the proper way. Or we could just bruth
        # force it. You know what we are gonna do..?
        suns_per_day = {
            'sunrise': [],
            'sunset': []
        }

        local_tz = pytz.timezone(row.timezone)

        # Add this when testing own timezones
        # x = row.geometry.centroid.x
        # if x < -7.5:
        #     tz = -60
        # elif x < 7.5:
        #     tz = 0
        # elif x < 22.5:
        #     tz = 60
        # else:
        #     tz = 120
        # local_tz = pytz.FixedOffset(tz)

        for d in dates:
            try:
                s = sun(location, date=d)
                suns_per_day['sunrise'].append(transform_date_to_winter_summer_time(s['sunrise'], local_tz))
                suns_per_day['sunset'].append(transform_date_to_winter_summer_time(s['sunset'], local_tz))
            except ValueError:
                pass

        # Now get the earliest sunrise for winter and summer time
        record = {
            'nuts_id': row['NUTS_ID'],
            'name': row['NAME_LATN'] if 'NAME_LATN' in row else 'hex',
            'timezone': str(local_tz),
            'sunrise': {
                'summer': find_earliest_and_latest_in_list(suns_per_day, 'sunrise', 'time_summer'),
                'winter': find_earliest_and_latest_in_list(suns_per_day, 'sunrise', 'time_winter'),
                'current': {
                    'earliest': min(suns_per_day['sunrise'], key=lambda x: x['local'].time())['local'].time(),
                    'earliest_day': min(suns_per_day['sunrise'], key=lambda x: x['local'].time())['local'].date(),
                    'latest': max(suns_per_day['sunrise'], key=lambda x: x['local'].time())['local'].time(),
                    'latest_day': max(suns_per_day['sunrise'], key=lambda x: x['local'].time())['local'].date()
                }
            },
            'sunset': {
                'summer': find_earliest_and_latest_in_list(suns_per_day, 'sunset', 'time_summer'),
                'winter': find_earliest_and_latest_in_list(suns_per_day, 'sunset', 'time_winter'),
                'current': {
                    'earliest': min(suns_per_day['sunset'], key=lambda x: x['local'].time())['local'].time(),
                    'earliest_day': min(suns_per_day['sunset'], key=lambda x: x['local'].time())['local'].date(),
                    'latest': max(suns_per_day['sunset'], key=lambda x: x['local'].time())['local'].time(),
                    'latest_day': max(suns_per_day['sunset'], key=lambda x: x['local'].time())['local'].date()
                }
            }
        }

        timedata.append(record)

# Store
json.dump(timedata, open(PATH_TIMEDATA, 'w', encoding='UTF-8'), ensure_ascii=False, indent=2, default=str)

100%|██████████| 6838/6838 [01:30<00:00, 75.98it/s]


## Calculate amount of light at certain hours

In [6]:
sample_date_range = pd.date_range('2025-01-01', '2025-12-31', freq='7D')
timedata_hourly = []

def calculate_sunrise_for_date(day, hour, minute, timeevent, location, local_tz):

    dt = day.replace(hour=hour, minute=minute, tzinfo=local_tz)

    if timeevent == 'winter':

        # Remove DST
        if dt.dst():
            dt += timedelta(hours=1)

    elif timeevent == 'summer':

        # Add DST
        if not dt.dst():
            dt -= timedelta(hours=1)
        
    # Calculate sunrise
    elevation = location.solar_elevation(dt)

    return {
        'time': f"{hour:02}:{minute:02}",
        'elevation': round(elevation, 1),
        'day': dt
    }

def calculate_sunrise_hours_for_event(location, local_tz, timeevent, earliest):
    records = []
    # loop time
    
    for hour in range(3, 15):
        for minute in [0, 30]:

            day_records = []

            # First loop through sample dates (only one per week)
            sample_results = []
            for day in sample_date_range:
                sample_results.append(calculate_sunrise_for_date(day, hour, minute, timeevent, location, local_tz))

            # Find the lowest elevation
            sample_results.sort(key=lambda x: x['elevation'])
            if earliest:
                sample_day = sample_results[-1]
            else:
                sample_day = sample_results[0]

            more_granulare_dates = pd.date_range(sample_day['day'] - timedelta(days=8), sample_day['day'] + timedelta(days=8), freq='D')
            more_granulare_dates = more_granulare_dates[(more_granulare_dates >= '2025-01-01') & (more_granulare_dates <= '2025-12-31')]

            # Find lowest sun elevation in all days.
            for day in more_granulare_dates:
                day_records.append(calculate_sunrise_for_date(day, hour, minute, timeevent, location, local_tz))

            # Now find the lowest elevation
            day_records.sort(key=lambda x: x['elevation'])
            if earliest:
                records.append(day_records[-1])
            else:
                records.append(day_records[0])
            

    return records

def calculate_sunset_for_date(day, hour, minute, timeevent, location, local_tz):

    dt = day.replace(hour=hour, minute=minute, tzinfo=local_tz)

    if timeevent == 'winter':

        # Remove DST
        if dt.dst():
            dt += timedelta(hours=1)

    elif timeevent == 'summer':

        # Add DST
        if not dt.dst():
            dt -= timedelta(hours=1)

    elif timeevent == 'current':
        pass

    else:
        raise ValueError(f"Unknown timeevent {timeevent}")
        
    # Calculate sunrise
    elevation = location.solar_elevation(dt)

    return {
        'time': f"{hour:02}:{minute:02}",
        'elevation': round(elevation, 1),
        'day': dt
    }


def calculate_sunset_hours_for_event(location, local_tz, timeevent, earliest):
    records = []
    # loop time


    for hour in range(13, 24):
        for minute in [0, 30]:    
    
            # First loop through sample dates (only one per week)
            sample_results = []
            for day in sample_date_range:
                sample_results.append(calculate_sunset_for_date(day, hour, minute, timeevent, location, local_tz))

            # Find the lowest elevation
            sample_results.sort(key=lambda x: x['elevation'])
            if earliest:
                sample_day = sample_results[0]
            else:
                sample_day = sample_results[-1]

            more_granulare_dates = pd.date_range(sample_day['day'] - timedelta(days=8), sample_day['day'] + timedelta(days=8), freq='D')
            more_granulare_dates = more_granulare_dates[(more_granulare_dates >= '2025-01-01') & (more_granulare_dates <= '2025-12-31')]

            # Find lowest sun elevation in all days.
            day_records = []
            for day in more_granulare_dates:
                day_records.append(calculate_sunset_for_date(day, hour, minute, timeevent, location, local_tz))

            # Now find the highest elevation
            day_records.sort(key=lambda x: x['elevation'])

            if earliest:
                records.append(day_records[0])
            else:
                records.append(day_records[-1])

    return records


with tqdm(total=len(gdf_grid)) as pbar:
    for cell in timedata:

        pbar.update()

        # if cell['nuts_id'] != 4502:
        #     continue

        local_tz = ZoneInfo(cell['timezone'])

        # Get Grid Cell and Location Object
        grid_cell = gdf_grid[gdf_grid.NUTS_ID == cell['nuts_id']].iloc[0]
        x = grid_cell.geometry.centroid.y
        y = grid_cell.geometry.centroid.x  
        location = Location(LocationInfo(None, None, None, x, y))

        record = {
            'nuts_id': cell['nuts_id'],
            'sunrise': {
                # 'current': {
                #     'latest': calculate_sunrise_hours_for_event(location, local_tz, 'current', False)
                # },
                'winter': {
                    'latest': calculate_sunrise_hours_for_event(location, local_tz, 'winter', False),
                    'earliest': calculate_sunrise_hours_for_event(location, local_tz, 'winter', True)
                },
                'summer': {
                    'latest': calculate_sunrise_hours_for_event(location, local_tz, 'summer', False),
                    'earliest': calculate_sunrise_hours_for_event(location, local_tz, 'summer', True)
                }
            },
            'sunset': {
                # 'current': {
                #     'earliest': calculate_sunset_hours_for_event(location, local_tz, 'current', True)
                # },
                'winter': {
                    'earliest': calculate_sunset_hours_for_event(location, local_tz, 'winter', True),
                    'latest': calculate_sunset_hours_for_event(location, local_tz, 'winter', False)
                },
                'summer': {
                    'earliest': calculate_sunset_hours_for_event(location, local_tz, 'summer', True),
                    'latest': calculate_sunset_hours_for_event(location, local_tz, 'summer', False)
                }
            }
        }

        timedata_hourly.append(record)

# Store
json.dump(timedata_hourly, open(consts.PATH_TIMEDATA_HOURLY_HEX, 'w', encoding='UTF-8'), ensure_ascii=False, default=str)

100%|██████████| 6838/6838 [23:57<00:00,  4.76it/s]


## Example

In [ ]:
local_tz = pytz.timezone("Europe/Oslo")
dt = datetime(2025, 1, 4, 8, 30)
# dt = datetime(2025, 10, 25, 8, 30)
dt = local_tz.localize(dt)

x = 63.5183
y = 10.3139

location = LocationInfo(None, None, None, x, y)

l = Location(location)
e = l.solar_elevation(dt)
e

5.62188698721711

In [47]:
local_tz = pytz.timezone("Europe/Athens")
# dt = datetime(2025, 1, 4, 8, 30)
dt = datetime(2025, 10, 25, 8, 30)
dt = local_tz.localize(dt)

x = 38.70305348749065
y = 21.60567212232599

location = Observer(x, y)

s = sun(location, date=dt)
s


{'dawn': datetime.datetime(2025, 10, 25, 7, 25, 32, 389387, tzinfo=<DstTzInfo 'Europe/Athens' EEST+3:00:00 DST>),
 'sunrise': datetime.datetime(2025, 10, 25, 7, 53, 15, 227043, tzinfo=<DstTzInfo 'Europe/Athens' EEST+3:00:00 DST>),
 'noon': datetime.datetime(2025, 10, 25, 10, 17, 35, tzinfo=datetime.timezone.utc),
 'sunset': datetime.datetime(2025, 10, 25, 18, 41, 23, 994252, tzinfo=<DstTzInfo 'Europe/Athens' EEST+3:00:00 DST>),
 'dusk': datetime.datetime(2025, 10, 25, 19, 9, 5, 610408, tzinfo=<DstTzInfo 'Europe/Athens' EEST+3:00:00 DST>)}

## Latest sunrise per year

In [ ]:
h_s = ""
h_i = 10000

local_tz = pytz.timezone("Europe/Rome")
x, y = 45.0979, 10.3193

location = Observer(x, y)

l = Location(location)

sunrise_for_day = []
for date in dates:
    dt = local_tz.localize(date)

    try:
        s = sun(location, date=dt)
        sunrise = s['sunrise']
        if not sunrise.dst():
            sunrise += timedelta(hours=1)

        sunrise_for_day.append(sunrise)
    except: pass


max(sunrise_for_day, key=lambda x: x.time())

datetime.datetime(2025, 1, 2, 8, 57, 40, 925584, tzinfo=<DstTzInfo 'Europe/Rome' CET+1:00:00 STD>)

### Lowerst Sunposition for specific time

In [26]:
h_s = ""
h_i = 10000

local_tz = pytz.timezone("Europe/Rome")
x, y = 45.1238,10.6952

location = LocationInfo(None, None, None, x, y)

l = Location(location)

kalax = []

for date in dates:
    dt = local_tz.localize(date)

    dt = dt + timedelta(hours=20, minutes=30)

    # Ständig Winterzeit
    # if dt.dst():
    #     dt += timedelta(hours=1)

    # Ständig Sommerzeit
    if not dt.dst():
        dt -= timedelta(hours=1)

    e = l.solar_elevation(dt)

    kalax.append({
        'date': dt,
        'elevation': round(e, 1)
    })

    if e < h_i:
        h_i = e
        h_s = dt

kalax.sort(key=lambda x: x['elevation'])
kalax[0]

{'date': datetime.datetime(2025, 12, 1, 19, 30, tzinfo=<DstTzInfo 'Europe/Rome' CET+1:00:00 STD>),
 'elevation': -29.9}